# Trip Distribution Class

# Data
## The result from trip generation (trip production and attraction) will be passed to trip distribution

In [57]:

from IPython.display import HTML, display


class TripDistribution:

    def __init__(self, productions, attractions, travelTime, fare, income):
        self.productions = productions
        self.attractions = attractions
        self.travelTime = travelTime
        self.fare = fare
        self.income = income
        self.row = len(productions)
        self.col = len(attractions)
        self.possibleError = sum(productions) * 0.2
        self.error = 0

    def getGeneralizedCost(self, cost):
        return 1.0 / (cost * cost)
    
    def computeCost(self, travelTime, fare, income):
        costMatrix = [[1 for x in range(self.row)] for y in range(self.col)]
        for x in range(self.row):
            for y in range(self.col):
                costMatrix[x][y] = travelTime[x][y] * income[x] + fare[x][y]
        print(costMatrix)
        return costMatrix

    def getTripDistribution(self):
        distributions = [[self.attractions[y] for x in range(self.row)] for y in range(self.col)]
        #costMatrix = [[1 for x in range(self.row)] for y in range(self.col)]
        costMatrix = self.computeCost(self.travelTime, self.fare, self.income)
        A = [1 for x in range(self.row)]
        B = [1 for x in range(self.col)]

        currentBalancingFactor = 0  # 0 for A, 1 for B
        isConvergent = False

        while isConvergent == False:
            if currentBalancingFactor == 0:
                tempA = self.computeA(B, costMatrix)
                A = tempA
                currentBalancingFactor = 1
            elif currentBalancingFactor == 1:
                tempB = self.computeB(A, costMatrix)
                B = tempB
                currentBalancingFactor = 0
            distributions = self.computeDistributions(A, B, costMatrix)
            isConvergent = self.checkIfConvergent(distributions)
            print(str(A) + " " + str(B))
        return distributions

    def computeDistributions(self, A, B, costMatrix):
        distributions = [[self.attractions[y] for x in range(self.row)] for y in range(self.col)]
        for x in range(self.row):
            for y in range(self.col):
                distributions[x][y] = A[x] * self.productions[x] * B[y] * self.attractions[y] * self.getGeneralizedCost(costMatrix[x][y])
        return distributions

    def checkIfConvergent(self, distributions):
        error = self.getError(distributions)
        if error <= self.possibleError:
            self.error = error
            return True
        return False

    def getError(self, distributions):
        error = 0
        derivedProductions = [0 for x in range(self.row)]
        derivedAttractions = [0 for x in range(self.col)]

        for x in range(self.row):
            for y in range(self.col):
                derivedProductions[x] += distributions[x][y]
                derivedAttractions[y] += distributions[x][y]

        for x in range(self.row):
            error += abs(derivedProductions[x] - self.productions[x])
            error += abs(derivedAttractions[y] - self.attractions[y])

        return error
    
    
    

    def computeA(self, B, costMatrix):
        A = [1 for x in range(self.row)]
        for x in range(0, self.row):
            sum = 0.0
            for y in range(0, self.col):
                sum += B[y] * self.attractions[y] * self.getGeneralizedCost(costMatrix[x][y])
            A[x] = 1.0 / sum
        return A

    def computeB(self, A, costMatrix):
        B = [1 for x in range(self.col)]
        for x in range(0, self.row):
            sum = 0.0
            for y in range(0, self.col):
                sum += A[y] * self.productions[y] * self.getGeneralizedCost(costMatrix[x][y])
            B[x] = 1.0 / sum
        return B

In [58]:
def computeYearlyToHourlyRate(salary):
    return salary/(240.00 * 8)

productions = [4087439.65, 2814510.63]
attractions = [4358277.38, 2543675.17]
fares = [[21.98, 43.96], [45.84, 22.92]]
travelTime = [[1.015, 2.03], [2.0, 1.0]]
salary = [computeYearlyToHourlyRate(414109.7), computeYearlyToHourlyRate(252283.04)];
print(salary)




td = TripDistribution(productions, attractions, travelTime, fares, salary)

print("Productions: " + str(productions))
print("Attractions: " + str(attractions))

[215.68213541666668, 131.39741666666666]
Productions: [4087439.65, 2814510.63]
Attractions: [4358277.38, 2543675.17]


The following formula, which is from a book, Introduction to Transportation Engineering by Tom V. Mathew and K V Krishna Rao, is used:

$$T_{ij} = A_iO_iB_jD_jf(c_{ij})$$

Where $T_{ij}$ is the trips made from zone i to the zone j, $O_i$ refers to the trip productions made from zone i, $D_j$ refers to the trip attractions made from zone j, $c_{ij}$ is the general cost used from zone i to zone j, $A_i$ and $B_i$ are the balancing factor and the following formula are used:

 $$A_i = 1 / \sum_{j} B_{j}D_jf(c_{ij})$$
 $$B_i = 1 / \sum_{i} A_{i}O_if(c_{ij})$$
 
At first, one of the balancing factors will be set to 1s and the other will be compupted using the formula above. This will be done alternately until the some of all the origins or the destinations of the computed distributions will be close to the original productions and attractions. Lastly, the function $f(c_{ij})$ has the following formula:
$$f(c_{ij}) = 1/c_{ij}^2$$



In [60]:
distribution = td.getTripDistribution()

display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in distribution)
        )
))

[[240.89736744791665, 481.7947348958333], [308.6348333333333, 154.31741666666665]]
[0.011619796187198448, 0.0065544350203673545] [1, 1]


3566979.5805783635,520460.0694216368
844040.2210652309,1970470.4089347692


# Error


In [61]:
print("Error: " + str(td.error))

Error: 105489.38328718813
